# Numerical Optimization (CS215300) Assignment 3
## Introduction
In this assignment, we expect you to be able to solve constrained optimization problem by Scipy library. We want you to apply two algorithms on the following problem as benchmark, survey the method used in these libraries, and analysis the behaviour of these algorithms.
The library document link: https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html

## Task
1. Please solve the following problrem by using trust-constr method:
$$\begin{array}{lll}
        \min_{x_1,x_2} & f(x_1,x_2)=-x_1-x_2 \\
        \mbox{s.t. } & -2x_1^4 + 8x_1^3 -8x_1^2 + x_2 - 2 \le 0  \\
         & -4x_1^4 + 32x_1^3 - 88x_1^2 + 96x_1 + x_2 -36 \le 0   \\
         & 0 \le x_1 \le 3 \\
         & 0 \le x_2 \le 4 \\
\end{array}$$
2. Please use COBYLA method to solve the same problem.
3. In your report, please read the paper cited in the libraries, which gives the details of the algorithms. Write a brief introduction of the algorithms, and compare their behaviours in this benchmark. You are not necessarily to read the original paper, other resourses (ex. slides from other schools or surveys) are also acceptable, please include the link or paper name in your reference if you referred other resources.
4. Please provide the Jacobian and Hessian function in matrix form in your report. Basic latex syntax is supported in Markdown.
5. Rename this notebook file with your student ID and upload it to eeclass platform. (ex. 107xxxxxx.ipynb)

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from scipy.optimize import Bounds
from scipy.optimize import NonlinearConstraint
from scipy.optimize import minimize

### Define objective function

In [3]:
def f(x):
    # TODO
    return -x[0]-x[1]

### Define constraint functions and derivatives
Note: Please not use sparse matrices.

In [4]:
# TODO: derive and define the functions
def cons_f(x):
    # TODO
    return [-2*x[0]**4 +  8*x[0]**3 -  8*x[0]**2           + x[1] -  2,
            -4*x[0]**4 + 32*x[0]**3 - 88*x[0]**2 + 96*x[0] + x[1] - 36]
    
def cons_Jacobian(x):
    # TODO
    return [[ -8*x[0]**3 + 24*x[0]**2 -  16*x[0]     , 1],
            [-16*x[0]**3 + 96*x[0]**2 - 176*x[0] + 96, 1]]

def cons_Hessian(x, v):
    # TODO
    return v[0]*np.array([[ -24*x[0]**2 +  48*x[0] -  16, 0],  [0, 0]]) +\
           v[1]*np.array([[ -48*x[0]**2 + 192*x[0] - 176, 0],  [0, 0]])
            

# TODO: Insert the functions above into a NonlinearConstraint obeject
nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, 0, jac=cons_Jacobian, hess=cons_Hessian)

### Define the bounds 

In [5]:
# TODO: define the bounds
bounds = Bounds([0, 0], [3.0, 4.0])

### Call the minimize library

In [6]:
fun_der = lambda x: np.array([-1.0, -1.0])
fun_hess = lambda x: np.zeros((2,2))

x0 = np.array([0., 0.])

res = minimize(f, x0, method='trust-constr', jac=fun_der, hess=fun_hess,
               constraints=[nonlinear_constraint],bounds=bounds)

### Print out the result you get

In [7]:
print(res.x)

[0.61157004 3.44188615]


### Apply COBYLA method
* COBYLA doesn't support variable bounds explicitly, but we can formulate the bounds in the context of the genral constraints.
    * ref.: https://stackoverflow.com/questions/12781622/does-scipys-minimize-function-with-method-cobyla-accept-bounds

In [12]:
# TODO: Insert the functions above into a NonlinearConstraint obeject
nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, 0, jac=cons_Jacobian)

#lower and upper bound for variables
bounds = [[0, 3.0], [0, 4.0]]

cons = []
for factor in range(len(bounds)):
    lower, upper = bounds[factor]
    l = {'type': 'ineq',
         'fun': lambda x, lb=lower, i=factor: x[i] - lb}
    u = {'type': 'ineq',
         'fun': lambda x, ub=upper, i=factor: ub - x[i]}
    cons.append(l)
    cons.append(u)

cons.append(nonlinear_constraint)

res = minimize(f, x0, method='COBYLA', 
               constraints=cons)
print(res.x)

[0.61160344 3.44210482]


## Report

###  Type your report here. 中英文皆可
Method COBYLA uses the Constrained Optimization BY Linear Approximation (COBYLA) method [9], [10], [11]. The algorithm is based on linear approximations to the objective function and each constraint. The method wraps a FORTRAN implementation of the algorithm. The constraints functions ‘fun’ may return either a single number or an array or list of numbers.